# 🤖 MuJoCo进阶教程 (Part 2) - 机器人与夹爪
# MuJoCo Tutorial Part 2 - Robots and Grippers

---

## 🎯 学习目标

通过本教程，你将学会：

1. ✅ 理解关节(Joint)和执行器(Actuator)的概念
2. ✅ 创建简单的机器人结构
3. ✅ 添加夹爪机构
4. ✅ 控制机器人运动
5. ✅ 实现夹爪开合
6. ✅ 物体抓取仿真
7. ✅ 使用真实机器人模型(PAROL6)
8. ✅ 传感器和反馈

---

## 📖 目录

1. [环境准备](#1-环境准备)
2. [关节基础](#2-关节基础)
3. [执行器和控制](#3-执行器和控制)
4. [创建简单机械臂](#4-创建简单机械臂)
5. [夹爪机构](#5-夹爪机构)
6. [夹爪控制](#6-夹爪控制)
7. [物体抓取](#7-物体抓取)
8. [使用PAROL6机器人](#8-使用parol6机器人)
9. [综合项目：抓取任务](#9-综合项目抓取任务)

---

**前置要求**: 请先完成 Part 1 (10.mujoco_tutorial_basics.ipynb)

## 1. 环境准备
## 1. Environment Setup

In [ ]:
# 📦 导入必要的库
import os
import numpy as np
import mujoco
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time

# 设置环境
os.environ['MUJOCO_GL'] = 'egl'
os.environ['DISPLAY'] = ':0'

# 创建教程目录
tutorial_dir = './tutorial_robots'
os.makedirs(tutorial_dir, exist_ok=True)

print("✅ 环境设置完成！")
print(f"   MuJoCo版本: {mujoco.__version__}")
print(f"   工作目录: {tutorial_dir}")

---

## 2. 关节基础
## 2. Joint Basics

### 🔗 关节类型

MuJoCo支持多种关节类型：

| 关节类型 | 自由度 | 说明 | 应用 |
|---------|-------|------|------|
| `hinge` | 1 | 旋转关节 | 门铰链、机械臂关节 |
| `slide` | 1 | 滑动关节 | 抽屉、夹爪手指 |
| `ball` | 3 | 球关节 | 肩关节、万向节 |
| `free` | 6 | 自由关节 | 自由落体物体 |

### 📐 关节坐标系

```
        Z (axis)
        |
        |  
        |______ Y
       /
      /
     X
     
hinge joint 绕Z轴旋转
slide joint 沿Z轴滑动
```

In [ ]:
# 🔗 示例1: 不同类型的关节
# Example 1: Different joint types

xml_joints = """
<mujoco model="joint_types">
    <option timestep="0.001"/>
    
    <worldbody>
        <geom name="ground" type="plane" size="3 3 0.1" rgba="0.8 0.8 0.8 1"/>
        <light pos="0 0 3" dir="0 0 -1"/>
        
        <!-- Hinge关节：旋转关节 -->
        <body name="hinge_base" pos="-1.5 0 0.5">
            <geom type="cylinder" size="0.05 0.1" rgba="0.5 0.5 0.5 1"/>
            
            <body name="hinge_link" pos="0 0 0.15">
                <joint name="hinge_joint" type="hinge" axis="0 0 1" 
                       range="-90 90" damping="0.1"/>
                <geom type="box" size="0.3 0.05 0.05" rgba="1 0 0 1"/>
            </body>
        </body>
        
        <!-- Slide关节：滑动关节 -->
        <body name="slide_base" pos="0 0 0.3">
            <geom type="box" size="0.05 0.05 0.3" rgba="0.5 0.5 0.5 1"/>
            
            <body name="slide_link" pos="0 0 0.35">
                <joint name="slide_joint" type="slide" axis="0 0 1" 
                       range="0 0.4" damping="1"/>
                <geom type="box" size="0.08 0.08 0.05" rgba="0 1 0 1"/>
            </body>
        </body>
        
        <!-- Free关节：自由运动 -->
        <body name="free_object" pos="1.5 0 1">
            <freejoint/>
            <geom type="sphere" size="0.1" density="1000" 
                  friction="0.8 0.1 0.1" rgba="0 0 1 1"/>
        </body>
    </worldbody>
</mujoco>
"""

# 保存并加载
xml_path = os.path.join(tutorial_dir, '01_joint_types.xml')
with open(xml_path, 'w') as f:
    f.write(xml_joints)

model_joints = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path))
data_joints = mujoco.MjData(model_joints)

print("✅ 关节类型演示场景创建成功！")
print(f"\n🔗 关节信息:")
print(f"   总关节数: {model_joints.njnt}")
print(f"   自由度(DOF): {model_joints.nv}")

# 打印关节名称
for i in range(model_joints.njnt):
    jnt_name = mujoco.mj_id2name(model_joints, mujoco.mjtObj.mjOBJ_JOINT, i)
    jnt_type = model_joints.jnt_type[i]
    type_names = ['free', 'ball', 'slide', 'hinge']
    print(f"   {i}: {jnt_name} - {type_names[jnt_type]}")

In [ ]:
# 🎬 渲染关节场景
renderer = mujoco.Renderer(model_joints, height=480, width=640)
renderer.update_scene(data_joints)
img = renderer.render()

plt.figure(figsize=(12, 8))
plt.imshow(img)
plt.title('不同类型的关节', fontsize=16)
plt.axis('off')
plt.show()

print("从左到右:")
print("   🔴 红色：Hinge关节（旋转）")
print("   🟢 绿色：Slide关节（滑动）")
print("   🔵 蓝色：Free关节（自由落体）")

---

## 3. 执行器和控制
## 3. Actuators and Control

执行器(Actuator)是对关节施加力或力矩的装置，类似于电机。

### ⚙️ 执行器类型

| 类型 | 控制量 | 说明 |
|------|--------|------|
| `motor` | 力/力矩 | 直接力控制 |
| `position` | 位置 | 位置伺服控制（最常用） |
| `velocity` | 速度 | 速度控制 |

### 🎮 控制参数

```xml
<position joint="my_joint" 
          kp="1000"          <!-- 位置增益 -->
          ctrlrange="-90 90" <!-- 控制范围（度） -->
          forcerange="-100 100"/> <!-- 力范围 -->
```

In [ ]:
# ⚙️ 示例2: 添加执行器
# Example 2: Adding actuators

xml_actuators = """
<mujoco model="actuators_demo">
    <option timestep="0.002"/>
    
    <worldbody>
        <geom name="ground" type="plane" size="2 2 0.1" rgba="0.8 0.8 0.8 1"/>
        <light pos="0 0 3" dir="0 0 -1"/>
        
        <!-- 简单的摆 -->
        <body name="base" pos="0 0 0.5">
            <geom type="cylinder" size="0.05 0.1" rgba="0.5 0.5 0.5 1"/>
            
            <!-- 连杆 -->
            <body name="link" pos="0 0 0.15">
                <joint name="pendulum" type="hinge" axis="1 0 0" 
                       range="-180 180" damping="0.5"/>
                <geom type="capsule" size="0.03 0.2" 
                      fromto="0 0 0 0 0 -0.4" rgba="1 0.5 0 1"/>
                <geom type="sphere" size="0.08" pos="0 0 -0.4" 
                      density="2000" rgba="1 0 0 1"/>
            </body>
        </body>
        
        <camera name="view" pos="1 1 1" xyaxes="-1 1 0 -0.5 -0.5 1" fovy="50"/>
    </worldbody>
    
    <!-- 添加位置控制执行器 -->
    <actuator>
        <position name="pendulum_actuator" joint="pendulum" 
                  kp="100" ctrlrange="-180 180" 
                  forcerange="-50 50"/>
    </actuator>
</mujoco>
"""

xml_path_act = os.path.join(tutorial_dir, '02_actuators.xml')
with open(xml_path_act, 'w') as f:
    f.write(xml_actuators)

model_act = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_act))
data_act = mujoco.MjData(model_act)

print("✅ 执行器演示场景创建成功！")
print(f"   执行器数量: {model_act.nu}")
print(f"   控制维度: {len(data_act.ctrl)}")

In [ ]:
# 🎮 控制执行器运动
# Control actuator motion

print("🎮 开始执行器控制演示...\n")

renderer_act = mujoco.Renderer(model_act, height=400, width=400)

# 定义控制序列：左右摆动
control_sequence = [
    (0, "初始位置"),
    (np.deg2rad(45), "向右摆45°"),
    (np.deg2rad(-45), "向左摆45°"),
    (0, "回到中心"),
]

images = []
for target_angle, description in control_sequence:
    print(f"⏱️  {description}...")
    
    # 设置目标位置
    data_act.ctrl[0] = target_angle
    
    # 运行仿真让系统稳定
    for _ in range(500):
        mujoco.mj_step(model_act, data_act)
    
    # 渲染当前状态
    renderer_act.update_scene(data_act, camera='view')
    img = renderer_act.render()
    images.append((img, description))

# 显示控制序列
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, (img, desc) in enumerate(images):
    axes[idx].imshow(img)
    axes[idx].set_title(desc, fontsize=12)
    axes[idx].axis('off')

plt.suptitle('执行器控制序列', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("\n✅ 执行器控制演示完成！")
print("   关键点：通过设置 data.ctrl[i] 来控制执行器")

---

## 4. 创建简单机械臂
## 4. Create Simple Robot Arm

现在让我们创建一个简单的2自由度机械臂。

In [ ]:
# 🤖 示例3: 2自由度机械臂
# Example 3: 2-DOF robot arm

xml_robot_arm = """
<mujoco model="simple_arm">
    <option timestep="0.002" gravity="0 0 -9.81"/>
    
    <asset>
        <material name="arm_mat" rgba="0.3 0.5 0.8 1" 
                  specular="0.5" shininess="0.3"/>
    </asset>
    
    <worldbody>
        <geom name="ground" type="plane" size="2 2 0.1" rgba="0.7 0.7 0.7 1"/>
        <light pos="1 1 2" dir="-0.5 -0.5 -1"/>
        
        <!-- 机械臂底座 -->
        <body name="base" pos="0 0 0.2">
            <geom type="cylinder" size="0.1 0.2" rgba="0.3 0.3 0.3 1"/>
            
            <!-- 第一连杆 -->
            <body name="link1" pos="0 0 0.2">
                <joint name="joint1" type="hinge" axis="0 0 1" 
                       range="-180 180" damping="1"/>
                <geom type="capsule" size="0.04 0.25" 
                      fromto="0 0 0 0 0 0.5" material="arm_mat"/>
                
                <!-- 第二连杆 -->
                <body name="link2" pos="0 0 0.5">
                    <joint name="joint2" type="hinge" axis="1 0 0" 
                           range="-120 120" damping="1"/>
                    <geom type="capsule" size="0.035 0.2" 
                          fromto="0 0 0 0 0 0.4" material="arm_mat"/>
                    
                    <!-- 末端 -->
                    <body name="end_effector" pos="0 0 0.4">
                        <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                        <site name="ee_site" pos="0 0 0" size="0.01"/>
                    </body>
                </body>
            </body>
        </body>
        
        <camera name="side" pos="1.5 0 0.8" xyaxes="0 1 0 -0.5 0 1" fovy="45"/>
        <camera name="front" pos="0 1.5 0.8" xyaxes="-1 0 0 0 -0.5 1" fovy="45"/>
    </worldbody>
    
    <!-- 添加执行器 -->
    <actuator>
        <position name="act1" joint="joint1" kp="100" 
                  ctrlrange="-180 180" forcerange="-50 50"/>
        <position name="act2" joint="joint2" kp="100" 
                  ctrlrange="-120 120" forcerange="-50 50"/>
    </actuator>
</mujoco>
"""

xml_path_arm = os.path.join(tutorial_dir, '03_simple_arm.xml')
with open(xml_path_arm, 'w') as f:
    f.write(xml_robot_arm)

model_arm = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_arm))
data_arm = mujoco.MjData(model_arm)

print("✅ 简单机械臂创建成功！")
print(f"\n🤖 机械臂信息:")
print(f"   关节数: {model_arm.njnt}")
print(f"   自由度: {model_arm.nv}")
print(f"   执行器: {model_arm.nu}")
print(f"   控制维度: {len(data_arm.ctrl)}")

In [ ]:
# 🎮 控制机械臂运动
# Control robot arm motion

renderer_arm = mujoco.Renderer(model_arm, height=400, width=400)

# 定义几个关键姿态
poses = [
    ([0, 0], "初始姿态"),
    ([np.deg2rad(45), np.deg2rad(30)], "姿态1"),
    ([np.deg2rad(90), np.deg2rad(60)], "姿态2"),
    ([np.deg2rad(-45), np.deg2rad(-30)], "姿态3"),
]

pose_images = []
for angles, desc in poses:
    # 设置控制输入
    data_arm.ctrl[:] = angles
    
    # 运行仿真
    for _ in range(500):
        mujoco.mj_step(model_arm, data_arm)
    
    # 渲染
    renderer_arm.update_scene(data_arm, camera='side')
    img = renderer_arm.render()
    pose_images.append((img, desc))

# 显示不同姿态
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, (img, desc) in enumerate(pose_images):
    axes[idx].imshow(img)
    axes[idx].set_title(desc, fontsize=12)
    axes[idx].axis('off')

plt.suptitle('机械臂不同姿态', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("✅ 机械臂运动控制完成！")

---

## 5. 夹爪机构
## 5. Gripper Mechanism

夹爪通常使用**平行夹爪**设计，由两个对称的手指组成。

### 🔧 夹爪设计要点

1. **Slide关节**: 手指使用滑动关节
2. **对称运动**: 两个手指朝相反方向移动
3. **摩擦力**: 增加摩擦系数以便抓握
4. **碰撞检测**: 确保手指能与物体接触

```
     ┌─────┐
     │ 基座 │
     └──┬──┘
   ◄────┼────►  (slide joints)
   │    │    │
   │    │    │  手指
   │    │    │
   └────┴────┘
```

In [ ]:
# 🤏 示例4: 简单夹爪
# Example 4: Simple gripper

xml_gripper = """
<mujoco model="simple_gripper">
    <option timestep="0.002" gravity="0 0 -9.81"/>
    
    <worldbody>
        <geom name="ground" type="plane" size="2 2 0.1" rgba="0.7 0.7 0.7 1"/>
        <light pos="0 0 2" dir="0 0 -1"/>
        
        <!-- 夹爪基座 -->
        <body name="gripper_base" pos="0 0 0.5">
            <geom name="base" type="box" size="0.05 0.05 0.1" 
                  rgba="0.3 0.3 0.3 1"/>
            
            <!-- 左手指 -->
            <body name="left_finger" pos="0 0 -0.05">
                <joint name="left_joint" type="slide" axis="0 1 0" 
                       range="0 0.08" damping="10"/>
                <geom name="left_finger_geom" type="box" 
                      size="0.015 0.01 0.06" 
                      pos="0 0.03 0" 
                      friction="1.5 0.1 0.1"
                      rgba="0.8 0.3 0.3 1"/>
            </body>
            
            <!-- 右手指 -->
            <body name="right_finger" pos="0 0 -0.05">
                <joint name="right_joint" type="slide" axis="0 1 0" 
                       range="-0.08 0" damping="10"/>
                <geom name="right_finger_geom" type="box" 
                      size="0.015 0.01 0.06" 
                      pos="0 -0.03 0" 
                      friction="1.5 0.1 0.1"
                      rgba="0.3 0.3 0.8 1"/>
            </body>
        </body>
        
        <!-- 测试物体 -->
        <body name="test_object" pos="0 0 0.35">
            <freejoint/>
            <geom type="box" size="0.025 0.025 0.025" 
                  density="500" friction="1.0 0.1 0.1"
                  rgba="1 0.8 0 1"/>
        </body>
        
        <camera name="view" pos="0.6 0.6 0.6" xyaxes="-1 1 0 -0.5 -0.5 1" fovy="45"/>
    </worldbody>
    
    <!-- 夹爪执行器 -->
    <actuator>
        <position name="left_act" joint="left_joint" kp="1000" 
                  ctrlrange="0 0.08" forcerange="-100 100"/>
        <position name="right_act" joint="right_joint" kp="1000" 
                  ctrlrange="-0.08 0" forcerange="-100 100"/>
    </actuator>
</mujoco>
"""

xml_path_gripper = os.path.join(tutorial_dir, '04_simple_gripper.xml')
with open(xml_path_gripper, 'w') as f:
    f.write(xml_gripper)

model_gripper = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_gripper))
data_gripper = mujoco.MjData(model_gripper)

print("✅ 夹爪创建成功！")
print(f"\n🤏 夹爪信息:")
print(f"   手指数量: 2")
print(f"   关节类型: slide (滑动)")
print(f"   控制维度: {model_gripper.nu}")
print(f"   测试物体: 黄色立方体")

---

## 6. 夹爪控制
## 6. Gripper Control

夹爪控制的关键是协调两个手指的运动。

In [ ]:
# 🤏 夹爪开合控制
# Gripper open/close control

def set_gripper(data, width):
    """
    设置夹爪开合度
    
    参数:
        data: MjData对象
        width: 夹爪开口宽度 (0=完全闭合, 0.08=完全打开)
    """
    data.ctrl[0] = width / 2  # 左手指
    data.ctrl[1] = -width / 2  # 右手指（反向）

renderer_gripper = mujoco.Renderer(model_gripper, height=400, width=400)

# 测试夹爪开合序列
gripper_states = [
    (0.08, "完全打开"),
    (0.06, "部分打开"),
    (0.04, "部分闭合"),
    (0.01, "完全闭合"),
]

gripper_images = []
for width, desc in gripper_states:
    # 设置夹爪
    set_gripper(data_gripper, width)
    
    # 运行仿真让夹爪移动到位
    for _ in range(200):
        mujoco.mj_step(model_gripper, data_gripper)
    
    # 渲染
    renderer_gripper.update_scene(data_gripper, camera='view')
    img = renderer_gripper.render()
    gripper_images.append((img, desc, width))

# 显示夹爪状态
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, (img, desc, width) in enumerate(gripper_images):
    axes[idx].imshow(img)
    axes[idx].set_title(f"{desc}\n宽度={width*1000:.0f}mm", fontsize=12)
    axes[idx].axis('off')

plt.suptitle('夹爪开合状态', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("✅ 夹爪控制演示完成！")
print("\n💡 控制提示:")
print("   - 宽度=0: 夹爪完全闭合")
print("   - 宽度=0.08: 夹爪完全打开")
print("   - 左右手指对称移动")

---

## 7. 物体抓取
## 7. Object Grasping

现在让我们实现完整的抓取动作序列。

In [ ]:
# 🎯 物体抓取仿真
# Object grasping simulation

print("🎯 开始抓取序列仿真...\n")

# 重置场景
mujoco.mj_resetData(model_gripper, data_gripper)

# 抓取序列
grasp_sequence = [
    {"step": 0, "width": 0.08, "desc": "1. 初始状态：夹爪打开"},
    {"step": 100, "width": 0.08, "desc": "2. 物体下落中"},
    {"step": 200, "width": 0.08, "desc": "3. 物体到达夹爪中心"},
    {"step": 300, "width": 0.015, "desc": "4. 夹爪闭合，抓取物体"},
]

grasp_images = []
step_count = 0

for seq in grasp_sequence:
    target_step = seq["step"]
    
    # 运行到目标步数
    while step_count < target_step:
        set_gripper(data_gripper, seq["width"])
        mujoco.mj_step(model_gripper, data_gripper)
        step_count += 1
    
    # 渲染当前状态
    renderer_gripper.update_scene(data_gripper, camera='view')
    img = renderer_gripper.render()
    grasp_images.append((img, seq["desc"]))
    
    print(f"⏱️  {seq['desc']}")

# 继续仿真以观察抓取效果
print("\n⏱️  5. 保持抓取状态")
for _ in range(200):
    set_gripper(data_gripper, 0.015)
    mujoco.mj_step(model_gripper, data_gripper)

renderer_gripper.update_scene(data_gripper, camera='view')
final_img = renderer_gripper.render()
grasp_images.append((final_img, "5. 抓取成功！"))

# 显示抓取序列
fig = plt.figure(figsize=(15, 10))
for idx, (img, desc) in enumerate(grasp_images[:5]):
    ax = fig.add_subplot(2, 3, idx+1)
    ax.imshow(img)
    ax.set_title(desc, fontsize=11)
    ax.axis('off')

plt.suptitle('物体抓取序列', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("\n✅ 抓取仿真完成！")
print("   物体被成功抓取并固定在夹爪中")

---

## 8. 使用PAROL6机器人
## 8. Using PAROL6 Robot

现在让我们使用项目中已有的PAROL6真实机器人模型。

In [ ]:
# 🤖 加载PAROL6机器人
# Load PAROL6 robot

# 使用项目中的PAROL6场景
parol6_xml_path = './asset/example_scene_parol6.xml'
parol6_xml_abs = os.path.abspath(parol6_xml_path)

try:
    model_parol6 = mujoco.MjModel.from_xml_path(parol6_xml_abs)
    data_parol6 = mujoco.MjData(model_parol6)
    
    print("✅ PAROL6机器人加载成功！")
    print(f"\n🤖 PAROL6机器人信息:")
    print(f"   关节总数: {model_parol6.njnt}")
    print(f"   执行器数量: {model_parol6.nu}")
    print(f"   摄像头数量: {model_parol6.ncam}")
    
    # 列出关节名称
    print(f"\n🔗 关节列表:")
    for i in range(min(8, model_parol6.njnt)):  # 显示前8个关节
        jnt_name = mujoco.mj_id2name(model_parol6, mujoco.mjtObj.mjOBJ_JOINT, i)
        if jnt_name:
            print(f"   {i}: {jnt_name}")
    
    # 列出执行器
    print(f"\n⚙️ 执行器列表:")
    for i in range(model_parol6.nu):
        act_name = mujoco.mj_id2name(model_parol6, mujoco.mjtObj.mjOBJ_ACTUATOR, i)
        if act_name:
            print(f"   {i}: {act_name}")
            
except Exception as e:
    print(f"❌ 无法加载PAROL6: {e}")
    print("   请确保 asset/example_scene_parol6.xml 文件存在")

In [ ]:
# 🎥 渲染PAROL6场景
# Render PAROL6 scene

if 'model_parol6' in locals():
    renderer_parol6 = mujoco.Renderer(model_parol6, height=480, width=640)
    
    # 多视角展示
    camera_names = ['agentview', 'topview', 'sideview']
    parol6_views = []
    
    for cam_name in camera_names:
        renderer_parol6.update_scene(data_parol6, camera=cam_name)
        img = renderer_parol6.render()
        parol6_views.append((img, cam_name))
    
    # 显示
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, (img, name) in enumerate(parol6_views):
        axes[idx].imshow(img)
        axes[idx].set_title(f'{name}视角', fontsize=12)
        axes[idx].axis('off')
    
    plt.suptitle('PAROL6六轴机械臂 - 多视角', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("✅ PAROL6场景渲染完成！")
else:
    print("⚠️ PAROL6模型未加载，跳过渲染")

In [ ]:
# 🎮 PAROL6夹爪控制
# PAROL6 gripper control

if 'model_parol6' in locals():
    print("🤖 PAROL6夹爪控制演示\n")
    
    # PAROL6的夹爪是索引6和7的执行器
    def set_parol6_gripper(data, state):
        """
        设置PAROL6夹爪状态
        
        参数:
            data: MjData对象
            state: 'open' 或 'close'
        """
        if state == 'open':
            data.ctrl[6] = 0.04  # 左手指
            data.ctrl[7] = -0.04  # 右手指
        elif state == 'close':
            data.ctrl[6] = 0.0
            data.ctrl[7] = 0.0
    
    # 测试夹爪开合
    gripper_states_parol6 = [
        ('open', "夹爪打开"),
        ('close', "夹爪闭合"),
    ]
    
    parol6_gripper_images = []
    
    for state, desc in gripper_states_parol6:
        # 重置到初始姿态
        data_parol6.qpos[:6] = [0, 0, 0, 0, 0, 0]  # 机械臂关节角度
        
        # 设置夹爪
        set_parol6_gripper(data_parol6, state)
        
        # 运行仿真
        for _ in range(300):
            mujoco.mj_step(model_parol6, data_parol6)
        
        # 渲染（使用gripper_cam获得夹爪特写）
        try:
            renderer_parol6.update_scene(data_parol6, camera='gripper_cam')
        except:
            renderer_parol6.update_scene(data_parol6, camera='agentview')
        
        img = renderer_parol6.render()
        parol6_gripper_images.append((img, desc))
        
        print(f"⏱️  {desc}")
    
    # 显示对比
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    for idx, (img, desc) in enumerate(parol6_gripper_images):
        axes[idx].imshow(img)
        axes[idx].set_title(f'PAROL6 - {desc}', fontsize=14)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ PAROL6夹爪控制演示完成！")
else:
    print("⚠️ PAROL6模型未加载，跳过夹爪控制演示")

---

## 9. 综合项目：抓取任务
## 9. Comprehensive Project: Grasping Task

最后，让我们创建一个完整的抓取任务，结合机械臂和夹爪。

In [ ]:
# 🎯 综合项目：机械臂+夹爪抓取
# Comprehensive project: Arm + Gripper grasping

xml_complete = """
<mujoco model="complete_grasping">
    <option timestep="0.002" gravity="0 0 -9.81"/>
    
    <asset>
        <material name="arm_mat" rgba="0.3 0.5 0.8 1"/>
        <material name="gripper_mat" rgba="0.6 0.6 0.6 1"/>
    </asset>
    
    <worldbody>
        <geom name="ground" type="plane" size="2 2 0.1" rgba="0.7 0.7 0.7 1"/>
        <light pos="1 1 2" dir="-0.5 -0.5 -1"/>
        
        <!-- 桌子 -->
        <body name="table" pos="0.5 0 0">
            <geom type="box" size="0.4 0.3 0.3" pos="0 0 0.3" 
                  rgba="0.6 0.4 0.2 1"/>
        </body>
        
        <!-- 机械臂底座 -->
        <body name="base" pos="0 0 0.1">
            <geom type="cylinder" size="0.08 0.1" rgba="0.3 0.3 0.3 1"/>
            
            <!-- 连杆1 -->
            <body name="link1" pos="0 0 0.1">
                <joint name="j1" type="hinge" axis="0 0 1" 
                       range="-180 180" damping="1"/>
                <geom type="capsule" size="0.03 0.15" 
                      fromto="0 0 0 0 0 0.3" material="arm_mat"/>
                
                <!-- 连杆2 -->
                <body name="link2" pos="0 0 0.3">
                    <joint name="j2" type="hinge" axis="1 0 0" 
                           range="-120 120" damping="1"/>
                    <geom type="capsule" size="0.025 0.15" 
                          fromto="0 0 0 0.3 0 0" material="arm_mat"/>
                    
                    <!-- 夹爪基座 -->
                    <body name="gripper_base" pos="0.3 0 0" quat="0.707 0 0.707 0">
                        <geom type="box" size="0.03 0.03 0.06" 
                              material="gripper_mat"/>
                        
                        <!-- 左手指 -->
                        <body name="left_finger" pos="0 0 -0.04">
                            <joint name="left_j" type="slide" axis="0 1 0" 
                                   range="0 0.05" damping="10"/>
                            <geom type="box" size="0.01 0.008 0.04" 
                                  pos="0 0.02 0" friction="1.5 0.1 0.1"
                                  rgba="0.8 0.3 0.3 1"/>
                        </body>
                        
                        <!-- 右手指 -->
                        <body name="right_finger" pos="0 0 -0.04">
                            <joint name="right_j" type="slide" axis="0 1 0" 
                                   range="-0.05 0" damping="10"/>
                            <geom type="box" size="0.01 0.008 0.04" 
                                  pos="0 -0.02 0" friction="1.5 0.1 0.1"
                                  rgba="0.3 0.3 0.8 1"/>
                        </body>
                    </body>
                </body>
            </body>
        </body>
        
        <!-- 目标物体 -->
        <body name="target" pos="0.5 0 0.65">
            <freejoint/>
            <geom type="cylinder" size="0.02 0.03" 
                  density="500" friction="1.0 0.1 0.1"
                  rgba="1 0.5 0 1"/>
        </body>
        
        <camera name="overview" pos="1 1 1.2" xyaxes="-1 1 0 -0.5 -0.5 1" fovy="45"/>
        <camera name="side" pos="0 1.2 0.7" xyaxes="-1 0 0 0 -0.5 1" fovy="45"/>
    </worldbody>
    
    <actuator>
        <!-- 机械臂执行器 -->
        <position name="a1" joint="j1" kp="100" ctrlrange="-180 180"/>
        <position name="a2" joint="j2" kp="100" ctrlrange="-120 120"/>
        <!-- 夹爪执行器 -->
        <position name="a_left" joint="left_j" kp="1000" ctrlrange="0 0.05"/>
        <position name="a_right" joint="right_j" kp="1000" ctrlrange="-0.05 0"/>
    </actuator>
</mujoco>
"""

xml_path_complete = os.path.join(tutorial_dir, '05_complete_grasping.xml')
with open(xml_path_complete, 'w') as f:
    f.write(xml_complete)

model_complete = mujoco.MjModel.from_xml_path(os.path.abspath(xml_path_complete))
data_complete = mujoco.MjData(model_complete)

print("✅ 完整抓取系统创建成功！")
print(f"\n🎯 系统组成:")
print(f"   ✓ 2自由度机械臂")
print(f"   ✓ 2指平行夹爪")
print(f"   ✓ 桌面场景")
print(f"   ✓ 目标物体（橙色圆柱）")
print(f"\n📊 控制信息:")
print(f"   总执行器: {model_complete.nu}")
print(f"   控制维度: {len(data_complete.ctrl)}")

In [ ]:
# 🎬 执行完整的抓取任务
# Execute complete grasping task

print("🎬 开始执行抓取任务...\n")

renderer_complete = mujoco.Renderer(model_complete, height=480, width=640)

# 定义抓取序列
task_sequence = [
    {
        "name": "初始姿态",
        "arm": [0, 0],  # 机械臂关节角度
        "gripper": 0.05,  # 夹爪开度
        "steps": 100,
    },
    {
        "name": "移动到物体上方",
        "arm": [np.deg2rad(0), np.deg2rad(60)],
        "gripper": 0.05,
        "steps": 300,
    },
    {
        "name": "下降到抓取位置",
        "arm": [np.deg2rad(0), np.deg2rad(80)],
        "gripper": 0.05,
        "steps": 300,
    },
    {
        "name": "闭合夹爪",
        "arm": [np.deg2rad(0), np.deg2rad(80)],
        "gripper": 0.01,
        "steps": 200,
    },
    {
        "name": "抬起物体",
        "arm": [np.deg2rad(0), np.deg2rad(40)],
        "gripper": 0.01,
        "steps": 300,
    },
]

task_images = []

for task in task_sequence:
    print(f"⏱️  {task['name']}...")
    
    # 设置控制输入
    data_complete.ctrl[0:2] = task['arm']
    data_complete.ctrl[2] = task['gripper'] / 2
    data_complete.ctrl[3] = -task['gripper'] / 2
    
    # 运行仿真
    for _ in range(task['steps']):
        mujoco.mj_step(model_complete, data_complete)
    
    # 渲染
    renderer_complete.update_scene(data_complete, camera='overview')
    img = renderer_complete.render()
    task_images.append((img, task['name']))

# 显示任务序列
fig = plt.figure(figsize=(15, 10))
for idx, (img, desc) in enumerate(task_images):
    ax = fig.add_subplot(2, 3, idx+1)
    ax.imshow(img)
    ax.set_title(f"{idx+1}. {desc}", fontsize=11)
    ax.axis('off')

plt.suptitle('完整抓取任务序列', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("\n✅ 抓取任务完成！")
print("\n🎉 恭喜！你已经掌握了:")
print("   ✓ 机械臂控制")
print("   ✓ 夹爪操作")
print("   ✓ 物体抓取")
print("   ✓ 完整任务规划")

---

## 🎓 教程总结
## Tutorial Summary

### ✅ 你已经学会了：

1. **关节系统**
   - Hinge、Slide、Ball、Free关节
   - 关节范围和阻尼
   - 运动学链

2. **执行器控制**
   - Position、Velocity、Motor控制
   - 控制参数设置
   - 实时控制方法

3. **机械臂设计**
   - 串联机构
   - 多自由度协调
   - 工作空间

4. **夹爪机构**
   - 平行夹爪设计
   - 开合控制
   - 摩擦力设置

5. **物体抓取**
   - 抓取序列规划
   - 碰撞检测
   - 力控制

6. **真实机器人**
   - PAROL6六轴机械臂
   - 真实夹爪模型
   - 多视角观察

### 📁 本教程生成的文件

所有场景文件保存在 `./tutorial_robots/` 目录：
- `01_joint_types.xml` - 关节类型演示
- `02_actuators.xml` - 执行器控制
- `03_simple_arm.xml` - 简单机械臂
- `04_simple_gripper.xml` - 简单夹爪
- `05_complete_grasping.xml` - 完整抓取系统

### 🚀 实践建议

1. **练习1**: 增加机械臂的自由度（3DOF或4DOF）
2. **练习2**: 实现更复杂的抓取序列（放置、堆叠等）
3. **练习3**: 使用PAROL6完成真实的pick-and-place任务
4. **练习4**: 添加传感器反馈（力传感器、接触传感器）

### 📚 进阶主题

继续深入学习：
- 逆运动学(IK)求解
- 轨迹规划
- 力控制
- 视觉伺服
- 强化学习训练

### 🔗 相关资源

- [MuJoCo建模指南](https://mujoco.readthedocs.io/en/stable/modeling.html)
- [执行器参考](https://mujoco.readthedocs.io/en/stable/XMLreference.html#actuator)
- [PAROL6使用指南](./PAROL6_USAGE_GUIDE.md)

---

## 💡 下一步

你已经完成了MuJoCo的基础和进阶教程！

现在你可以：
1. 查看项目中的其他notebook（如 `8.smolvla_parol6.ipynb`）学习如何训练策略
2. 使用PAROL6采集演示数据
3. 训练自己的机器人控制策略
4. 部署到真实机器人上

---

🎉 **恭喜完成MuJoCo进阶教程！你已经掌握了机器人仿真的核心技能！**